In [216]:
import cv2
import numpy as np

In [217]:
def palette(img):
    """
    Return palette in descending order of frequency
    """
    arr = np.asarray(img)
    palette, index = np.unique(asvoid(arr).ravel(), return_inverse=True)
    palette = palette.view(arr.dtype).reshape(-1, arr.shape[-1])
    count = np.bincount(index)
    order = np.argsort(count)
    return palette[order[::-1]]

def asvoid(arr):
    """View the array as dtype np.void (bytes)
    This collapses ND-arrays to 1D-arrays, so you can perform 1D operations on them.
    http://stackoverflow.com/a/16216866/190597 (Jaime)
    http://stackoverflow.com/a/16840350/190597 (Jaime)
    Warning:
    >>> asvoid([-0.]) == asvoid([0.])
    array([False], dtype=bool)
    """
    arr = np.ascontiguousarray(arr)
    return arr.view(np.dtype((np.void, arr.dtype.itemsize * arr.shape[-1])))


img = cv2.imread("../dataset/upper_body/images/shirts_shirts/12142950cj_0_r.jpg")

C = palette(img)#[:50]

In [218]:
P = None
n = 0
for c in C:
    if P is None:
        P = np.array(c[None,:])
    else:
        for d in P:
            if ((c-d)**2).sum() > 50:
                P = np.concatenate([P,c[None,:]],axis=0)
                n += 1
                if n > 50:
                    break

C = P

In [219]:
from sklearn.cluster import KMeans, SpectralClustering

N_COLS = 5

model = KMeans(N_COLS)
model.fit(C)
C = model.cluster_centers_.astype(np.uint8)

In [220]:
import matplotlib

def hsv(arr, j):
    return matplotlib.colors.rgb_to_hsv(arr)[j]

P = np.zeros((200,N_COLS*100,3)).astype(np.uint8)
#for j in range(3):
j = 2
for c in range(N_COLS):
    for d in range(N_COLS):
        #if hsv(C[c], j) > hsv(C[d], j):
        if hsv(C[c], j) > hsv(C[d], j):
            tmp = C[c].copy()
            C[c] = C[d].copy()
            C[d] = tmp.copy()

for i, c in enumerate(C):
    #P[100*j:100*(1+j),100*i:100*(1+i),:] = c
    P[:,100*i:100*(1+i),:] = c


cv2.imshow("1",P)
cv2.waitKey()
cv2.destroyAllWindows()